In [ ]:
!pip install whisper_mic -q
!pip install --upgrade langchain -q
!pip install openai -q
!pip install docarray

In [18]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [19]:
from whisper_mic.whisper_mic import WhisperMic
import openai

In [20]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']
engine = os.environ['OPENAI_ENGINE']
model = os.environ['OPENAI_MODEL']

### base.pt, Pure OpenAI

Talk into the microphone in any language and wait for the LLM response.

In [8]:
mic = WhisperMic()
result = mic.listen()
print(result)

messages = [{"role": "user", "content": result}]
response = openai.ChatCompletion.create(model=model,engine=engine, messages=messages)
print(response.choices[0].message.content)

[11/27/23 13:18:17] INFO     No mic index provided, using default                                 ]8;id=882328;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=353992;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#54\54]8;;\

[11/27/23 13:18:19] INFO     Mic setup complete                                                   ]8;id=89379;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=338887;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#65\65]8;;\

                    INFO     Listening...                                                        ]8;id=501829;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=556908;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#160\160]8;;\

 Hello
Hello! How can I assist you today?


### base.en.pt, Pure OpenAI

Talk into the microphone in English and wait for the LLM response.

In [9]:
mic = WhisperMic(english=True)
result = mic.listen()
print(result)

messages = [{"role": "user", "content": result}]
response = openai.ChatCompletion.create(model=model,engine=engine, messages=messages)
print(response.choices[0].message.content)

100%|███████████████████████████████████████| 139M/139M [00:42<00:00, 3.43MiB/s]


[11/27/23 13:23:15] INFO     No mic index provided, using default                                 ]8;id=959949;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=869417;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#54\54]8;;\

[11/27/23 13:23:16] INFO     Mic setup complete                                                   ]8;id=9704;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=638998;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#65\65]8;;\

                    INFO     Listening...                                                        ]8;id=834278;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=118602;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#160\160]8;;\

 Hello.
Hi there! How can I assist you today?


### Langchain

In [21]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model=model,engine=engine)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


Change the FILEPATH variable to the path of the file you want to use as the document.

In [4]:
from langchain.document_loaders import TextLoader

FILEPATH = "testdoc.txt"

loader = TextLoader(FILEPATH)
docs = loader.load()
docs[0]

Document(page_content='Teknologi Generative AI dalam Pengerjaan Tugas Akhir\nPenulis: Muhammad Naufal Satriandana 13520068\nPada era modern ini, pesatnya perkembangan teknologi memunculkan banyak hal baru yang dapat memudahkan pekerjaan manusia. Teknologi AI menjadi salah satu teknologi yang sedang marak dan seringkali digunakan oleh manusia. Sebuah contoh dali teknologi AI adalah generative AI. Generative AI sudah terintegrasi ke banyak sekali aplikasi. Bahkan, aplikasi Microsoft word sekalipun sudah menggunakan generative AI untuk membantu dalam proses autocomplete. Sehingga, dengan mudahnya akses ke teknologi generative AI seperti ChatGPT, Github Copilot, dan Dall-e, pekerjaan manusia yang trivial seperti menyunting kesalahan penulisan dan merangkum teks, menjadi lebih mudah.\nPenggunaan generative AI yang dapat dilakukan salah satunya adalah pada pengerjaan tugas akhir. Mahasiswa yang sedang mengerjakan tugas akhir dapat menggunakan generative AI seperti ChatGPT untuk membantu dala

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
embeddings = OpenAIEmbeddings()

In [10]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [15]:
retriever = db.as_retriever()

In [30]:
from langchain.chains import RetrievalQA
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

Talk into the mic in any language and ask questions about the document.

In [41]:
mic = WhisperMic()
query = mic.listen()
print(query)

response = qa_stuff.run(query)
print(response)

[11/27/23 14:00:28] INFO     No mic index provided, using default                                 ]8;id=884495;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=888888;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#54\54]8;;\

[11/27/23 14:00:29] INFO     Mic setup complete                                                   ]8;id=110515;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=39205;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#65\65]8;;\

                    INFO     Listening...                                                        ]8;id=665399;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py\whisper_mic.py]8;;\:]8;id=680478;file://c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\whisper_mic\whisper_mic.py#160\160]8;;\

 Alapan judul dari Bacaan tersebut?


> Entering new RetrievalQA chain...

> Finished chain.
Judul dari bacaan tersebut adalah "Teknologi Generative AI dalam Pengerjaan Tugas Akhir".
